# 1.Up and running with TensorFlow

## 1.1 Setup

In [56]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

## 1.2 Creating_and_running_a_graph

In [57]:
import tensorflow as tf

tf.reset_default_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x*x*y + y + 2
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

꼭 이해해야할 중요한 점은 이코드가 뭔가 계산하는 것 같아 보이지만 실제로 어떤 계산도 수행하지 않습니다.  

이해해야 할 가장 중요한 점은 이 코드는 실제로는 실행되는 것처럼 보이지만 계산을 실제로 수행하지 않는다는 것입니다. 단지 계산 그래프를 생성합니다. 실제로 변수조차도 아직 초기화되지 않았습니다. 이 그래프를 평가하려면 TensorFlow 세션을 열고 이를 사용하여 변수를 초기화하고 f를 평가해야합니다. TensorFlow 세션은 CPU 및 GPU와 같은 장치에 작업을 배치하고 실행하며 모든 변수 값을 보유합니다. 다음 코드는 세션을 생성하고, 변수를 초기화하고, 평가한 다음, 세션을 닫습니다.

In [58]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


`sess.run()`을 항상 반복해야한다는 것은 다소 번거롭지만 다행스럽게도 더 좋은 방법이 있습니다.

In [59]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result    

42

**with** 블록의 세션은 기본 세션으로 설정됩니다.  
```python
x.initializer.run()
```
은  
```python
tf.get_default_session()
run (x.initializer)
```
호출하는 것과 동일하며, 마찬가지로 

```python
f.eval
```
은  

```python
tf.get_default_session()
run(f)
```
호출과 동일합니다. 이렇게하면 코드를 읽기 쉽게 만듭니다. 또한 세션이 블록 끝에서 자동으로 닫힙니다.  

단일 변수마다 `initializer`를 수동으로 실행하는 대신 `global_variables_initializer()` 함수를 사용할 수 있습니다. 실제로 초기화를 실제로 수행하지는 않지만 실행될 때 모든 변수를 초기화하는 노드를 그래프에 작성합니다.  

In [60]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
result    

42

Jupyter 내부 또는 Python 셸 내에서 InteractiveSession을 만드는 것이 더 좋습니다. 일반 세션과의 유일한 차이점은 InteractiveSession이 생성 될 때 자동으로 기본 세션으로 설정되므로 with 블록이 필요하지 않습니다.(그러나 세션이 끝나면 수동으로 세션을 닫아야 함).

In [61]:
init = tf.global_variables_initializer()

In [62]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


/home/ururu/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TensorFlow 프로그램은 일반적으로 두 부분으로 나뉩니다. 첫 번째 부분은 계산 그래프를 작성하고 (두 번째 부분은 실행 단계라고 함) 두 번째 부분은 실행 단계입니다. 구축 단계에서는 일반적으로 ML 모델 및 이를 계발하는 데 필요한 계산을 나타내는 계산 그래프를 작성합니다. 실행 단계는 일반적으로 학습 단계를 반복적으로 평가하는 루프를 실행하며(예: 미니 배치 당 하나의 단계) 점진적으로 모델 매개 변수를 향상시킵니다. 

## 1.3 Managing graphs

생성한 노드는 자동으로 기본 그래프에 추가됩니다.

In [63]:
tf.reset_default_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

대부분의 경우 괜찮습니다. 그러나 때때로 여러 개의 독립적인 그래프를 관리해야 할 수도 있습니다. 새 그래프를 만들고 일시적으로 기본 그래프를 with 블록 안에 만들면 다음과 같이할 수 있습니다.

In [64]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [65]:
x2.graph is tf.get_default_graph()

False

## 1.4 Life cycle of Node value

노드를 평가할 때 TensorFlow는 의존하는 노드 집합을 자동으로 결정하고 이 노드를 먼저 평가합니다.  

In [66]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


첫째, 이 코드는 매우 간단한 그래프를 정의합니다. 그런 다음 세션을 시작하고 그래프를 실행하여 y를 평가합니다. TensorFlow는 $y$가 $w$에 따라 달라지므로 $x$에 의존하므로 자동으로 $w$, $x$, $y$를 평가하고 $y$값을 반환합니다. 마지막으로 코드는 그래프를 실행하여 $z$를 평가합니다. 다시 한번, TensorFlow는 먼저 $w$와 $x$를 평가해야 함을 감지합니다. $w$와 $x$의 이전 평가 결과를 재사용하지 않는다는 점에 유의해야합니다. 요약하면 앞의 코드는 $w$와 $x$를 두 번 계산합니다.

모든 노드 값은 그래프 실행 사이에 세션에 의해 유지되는 변수 값을 제외하고 그래프 실행 사이에 삭제됩니다. 변수는 초기화될 때 일생을 시작하고 세션이 닫히면 끝납니다.

이전 코드와 같이 $w$와 $x$를 두 번 평가하지 않고 $y$와 $z$를 효율적으로 계산하려면 다음 코드와 같이 TensorFlow에게 한 번의 그래프 실행에서 $y$와 $z$를 모두 평가하도록 요청해야합니다.

In [67]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


## 1.5 Linear Regression

TensorFlow 연산(간단히 ops라고도 함)은 임의의 수의 입력을 가져와서 여러 출력을 생성할 수 있습니다. 예를 들어, 덧셈 및 곱셈 연산은 각각 2개의 입력을 취하여 하나의 출력을 생성합니다. 상수 및 변수에는 입력이 없습니다(소스 연산이라고 함). 입력과 출력은 텐서라고 불리는 다차원 배열입니다(따라서 "텐서 플로"이름). NumPy 배열과 마찬가지로 텐서는 유형과 모양을 가집니다. 사실, 파이썬 API에서 텐서는 단순히 NumPy ndarrays로 표현됩니다. 대개는 부동 소수점을 포함하지만 문자열 (임의의 바이트 배열)을 전달하는데도 사용할 수 있습니다.

지금까지의 예제에서, 텐서는 단지 하나의 스칼라 값을 포함하고 있었지만 물론 어떤 모양의 배열에 대해서도 계산을 수행 할 수 있습니다. 예를 들어, 다음 코드는 2차원 배열을 조작하여 캘리포니아 주택 데이터 세트에서 선형 회귀를 수행합니다. 데이터 집합을 가져 오는 것으로 시작합니다. 그러면 모든 훈련 샘플에 대한 입력 특성($x_{0} = 1$)이 추가됩니다.(NumPy를 사용하면 즉시 실행됩니다). 두 개의 TensorFlow 상수 노드 인 $X$와 $y$를 만들어이 데이터와 대상을 보유하고, TensorFlow에서 제공하는 행렬 연산을 사용하여 theta를 정의합니다. 이 행렬 함수는 `transpose()`, `matmul()` 및 `matrix_inverse()`인데, 이름만으로도 그 기능을 짐작할 수 있습니다.이 함수들은 계산을 수행하지 않습니다. 대신 그래프를 실행할 때 그래프를 수행할 노드를 생성합니다. theta의 정의는 Normal Equation $\hat{\theta} = (\mathbf{X}^{T} \cdot \mathbf{X})^{-1} \cdot \mathbf{X}^{T} \cdot \mathbf{y}$에 해당합니다. 마지막으로 코드는 세션을 만들고 theta를 평가합니다.

### 1.5.1 Using the Normal Equation

In [68]:
import numpy as np
from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [69]:
theta_value

array([[-3.7171074e+01],
       [ 4.3633682e-01],
       [ 9.3871783e-03],
       [-1.0717344e-01],
       [ 6.4540231e-01],
       [-4.1238391e-06],
       [-3.7809242e-03],
       [-4.2373490e-01],
       [-4.3720812e-01]], dtype=float32)

### 1.5.2 Compare with Numpy

In [70]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### 1.5.3 Compare with Scikit-Learn

In [71]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### 1.5.4 Using Batch Gradient Descent
#### 1.5.4.1 Manually computing the gradients

다음 코드는 몇 가지 새로운 요소를 제외하고는 상당히 이해하기 쉽습니다.  

- `random_uniform()` 함수는 NumPy의 `rand()` 함수와 마찬가지로 그 모양과 값의 범위에서 무작위 값을 포함하는 텐서를 생성하는 노드를 그래프에 만듭니다.  

- `assign()` 함수는 새로운 값을 변수에 할당할 노드를 만듭니다. 이 경우 **Batch Gradient Descent** 단계인 $\theta^{next\,step} = \theta -\eta  \triangledown_{\theta}MSE(\theta)$를 구현합니다.  

- 반복 루프는 훈련 단계를 반복 실행하고 (n_epochs 만큼) 100회 반복 할 때마다 현재 Mean Squared Error(mse)를 출력합니다. 매 반복마다 MSE가 내려가는 것을 볼 수 있습니다.

In [72]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [73]:
print(scaled_housing_data_plus_bias.mean(axis=0)) # 열방향
print(scaled_housing_data_plus_bias.mean(axis=1)) # 행방향
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [74]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print("")
    print(f"best_theta: \n{best_theta}")

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293705

best_theta: 
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44081801e-04]
 [-3.91945131e-02]
 [-8.61356556e-01]
 [-8.23479712e-01]]


<br>
<br>

#### 1.5.4.2 Using a `GradientDescentOptimizer`

In [75]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [76]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [77]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
Best theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


<br>
<br>

#### 1.5.4.3 Using a momentum optimizer

In [78]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [79]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

In [80]:
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [81]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)    
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Best theta:
[[ 2.068558  ]
 [ 0.8296167 ]
 [ 0.11875112]
 [-0.26552212]
 [ 0.30569226]
 [-0.00450316]
 [-0.03932616]
 [-0.8998917 ]
 [-0.87054664]]


## 1.6 Feeding data to the training algorithm

이전 코드를 수정하여 Mini-batch Gradient Descent를 구현해 보겠습니다. 이를 위해 매 반복마다 X와 Y를 다음 미니 배치로 대체해야 합니다. 가장 간단한 방법은 placeholder노드를 사용하는 것 입니다. 이 노드는 실제로 계산을 수행하지 않습니다. 실행 시 출력하도록 지정한 데이터만 출력합니다. 이들은 일반적으로 훈련 중에 TensorFlow에 훈련 데이터를 전달하는 데 사용됩니다. 실행시에 placeholder 값을 지정하지 않으면 예외가 발생합니다.

placeholder노드를 만들려면 placeholder() 함수를 호출하고 출력 텐서의 데이터 형식을 지정해야합니다. 선택적으로 적용하려는 경우 모양을 지정할 수도 있습니다. 차원에 **None**을 지정하면 "모든 크기"를 의미합니다. 예를 들어, 다음 코드는 플레이스홀더 노드 A와 노드 B를 만듭니다. B를 평가할 때 e_val A 메서드는 A의 값을 지정합니다. A는 랭크 2를 가져야하며(즉, 2 차원이어야 함) 3 개의 열이 있어야합니다. 그렇지 않으면 예외가 발생하지만 모든 행을 가질 수 있습니다.

### 1.6.1 Placeholder nodes

In [82]:
tf.reset_default_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[6. 7. 8.]]


In [83]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


### 1.6.2 Mini-batch Gradient Descent

In [84]:
n_epochs = 1000
learning_rate = 0.01

In [85]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [86]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [87]:
n_epochs = 10

In [88]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [89]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [90]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

<br>
<br>

## 1.7 Saving and restoring a model

모델을 훈련시키고 난 후에는 매개 변수를 디스크에 저장해야 원하는 때 언제든지 다시 돌아와 다른 프로그램에서 사용하고 다른 모델과 비교하는 등의 작업을 수행 할 수 있습니다. 또한 훈련 도중 일정한 간격으로 checkpoint을 저장하여 컴퓨터가 충돌하는 경우 처음부터 다시 시작하지 않고 마지막 검사 점부터 계속 실행할 수 있습니다.

TensorFlow는 모델 저장 및 복원을 매우 쉽게 만듭니다. 모든 변수 노드가 생성 된 후 생성 단계의 끝에서 `Saver 노드`를 만듭니다. 실행 단계에서 모델을 저장하고 체크 포인트 파일의 세션과 경로를 전달할 때마다 save() 메서드를 호출하기만 하면됩니다.

In [91]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")                                    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            
training_op = optimizer.minimize(mse)                                                 

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704


In [92]:
best_theta

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

모델을 복원하는 방법은 간단합니다. 이전 단계와 마찬가지로 작성 단계가 끝날 때 Saver를 작성하지만 실행 단계가 시작될 때 init 노드를 사용하여 변수를 초기화하는 대신 Saver 객체의 restore() 메소드를 호출합니다

In [93]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [94]:
np.allclose(best_theta, best_theta_restored)

True

기본적으로 Saver는 모든 변수를 자체 이름으로 저장 및 복원하지만 더 많은 제어가 필요한 경우 저장 또는 복원 할 변수와 사용할 이름을 지정할 수 있습니다. 예를 들어, 다음 Saver는 theta 변수만 weights란 이름으로 저장하고 복원할 것입니다.  

In [95]:
saver = tf.train.Saver({"weights": theta})

In [96]:
tf.reset_default_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [97]:
np.allclose(best_theta, best_theta_restored)

True

즉, 그래프를 작성하는 데 필요한 Python 코드가 없어도 사전 모델을 가져올 수 있습니다. 이는 모델을 계속 조정하고 저장할 때 매우 편리합니다. 모델을 빌드 한 코드의 버전을 검색하지 않고도 이전에 저장된 모델을 로드 할 수 있습니다.

## 1.8 Visualizing the graph

이제 Mini-batch Gradient Descent를 사용하여 선형 회귀 모델을 훈련시키는 계산 그래프를 얻었으며 규칙적인 간격으로 check point를 저장합니다. 일부 학습 통계를 제공하면 웹 브라우저에 이러한 통계를 대화형으로 시각적으로 표시 할 수 있습니다(예 : 학습 곡선). 또한 그래프의 정의를 제공할 수 있으며 이를 통해 탐색 할 수 있는 훌륭한 인터페이스를 제공합니다. 이는 그래프의 오류를 식별하고 병목 현상을 찾는 등의 작업에 매우 유용합니다.

첫 번째 단계는 프로그램을 약간 조정하여 TensorBoard에서 읽을 로그 디렉터리에 그래프 정의와 일부 학습 통계(예 : 교육 오류 (MSE))를 기록하는 것입니다. 프로그램을 실행할 때마다 다른 로그 디렉토리를 사용해야합니다. 그렇지 않으면 TensorBoard가 다른 실행의 통계를 병합하여 시각화를 엉망으로 만듭니다. 가장 간단한 해결책은 로그 디렉토리 이름에 시간을 포함시키는 것입니다. 프로그램 시작 부분에 다음 코드를 추가하면 됩니다.

In [98]:
tf.reset_default_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "logs/01_Up_and_running"
logdir = "{}/run-{}/".format(root_logdir, now)

In [99]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

그런 다음, 구성 단계의 끝에 다음 코드를 추가하십시오.

In [100]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [101]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

첫 번째 줄은 MSE 값을 평가하고 요약이라는 TensorBoard 호환 이진 로그 문자열에 쓰는 노드를 그래프에 만듭니다. 두 번째 줄은 로그 디렉토리의 로그 파일에 요약을 쓰는 데 사용할 FileWriter를 만듭니다. 첫 번째 매개 변수는 로그 디렉토리 경로를 나타냅니다 (이 경우 tf_logs / run-20190706091959 /와 같이 현재 디렉토리를 기준으로 함). 두 번째 매개 변수 (선택 사항)는 시각화하려는 그래프입니다. 생성시, FileWriter는 로그 디렉토리가 아직 존재하지 않는 경우 (그리고 필요한 경우 상위 디렉토리) 로그 파일을 작성하고 그래프 파일 정의를 이벤트 파일이라는 2진로그 파일에 작성합니다.

다음으로 훈련 도중 mse_summary 노드를 정기적으로 평가하도록 실행 단계를 업데이트해야합니다(예 : 10개 미니 배치). 그러면 file_writer를 사용하여 이벤트 파일에 기록 할 수있는 요약 정보가 출력됩니다. 다음은 업데이트 된 코드입니다.

In [102]:
with tf.Session() as sess:
    sess.run(init)        

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()    

In [103]:
file_writer.close()

In [104]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

<br>
<br>

## 1.9 Name scopes

신경망과 같은보다 복잡한 모델을 다룰 때 그래프는 수천 개의 노드로 인해 복잡하게 될 수 있습니다. 이를 피하기 위해 **이름 범위**를 만들 수 있습니다. 예를 들어 이전 코드를 수정하여 **"loss"**라는 이름 범위 내에서 **error** 및 **mse** 작업을 정의합니다.

In [106]:
tf.reset_default_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [107]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [108]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [109]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.070016  ]
 [ 0.8204561 ]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.3113402 ]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.8795008 ]]


In [110]:
print(error.op.name)

loss/sub


In [111]:
print(mse.op.name)

loss/mse


In [112]:
tf.reset_default_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


<br>
<br>

## 2.0 Modularity

두 개의 ReLU의 출력을 더하는 그래프를 만들고 싶다고 가정하면 ReLU는 입력의 선형 함수를 계산하고 양수이면 결과를 출력하고, 그렇지 않으면 0을 출력합니다.  

$$
    h_{ w,\,b }(\mathbf{X}) = max(\mathbf{X}\cdot \mathbf{w} \,+ b,\, 0)
$$

In [113]:
tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [114]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

더 좋은 코드 표현은 다음과 같이 작성할 수 있습니다.

In [115]:
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                   
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    
        b = tf.Variable(0.0, name="bias")                             
        z = tf.add(tf.matmul(X, w), b, name="z")                      
        return tf.maximum(z, 0., name="max")                          

In [116]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()